# FluidAINet

In [196]:
import numpy as np

## LoadData

In [75]:
curren_data, current_label = concat_data_label_all(TRAIN_FILES, 8, 3)

csv
csv
runtime: 3.9962249999999955


In [61]:
curren_data = np.asarray(curren_data)
current_label = np.asarray(current_label)

In [93]:
%run -i utils/preprocess.py

['/data/datasets/simulation_data/water/0/particles_2753.csv', '/data/datasets/simulation_data/water/0/particles_2559.csv']
csv
csv
runtime: 3.912877999999999
Fluid


In [95]:
voxel_index.shape

(62000, 3)

In [ ]:
%%writefile train.py


In [96]:
"""
get all files including screens and frames
"""
BASE_DIR = '/data/datasets/simulation_data'
DATA_DIR = os.path.join(BASE_DIR, 'water')

['1', '0']

In [118]:
def get_all_frames(data_dir=DATA_DIR):
    dirs = os.listdir(data_dir)
    frames = []
    for item in dirs:
        screen_path = os.path.join(data_dir, item)
        allfiles = os.listdir(screen_path)
        frames.extend(map(lambda x:os.path.join(item, x), allfiles))
    return map(lambda x:os.path.join(data_dir, x), frames)

In [123]:
TRAIN_FILES = get_all_frames(DATA_DIR)

In [126]:
for f in TRAIN_FILES:
    particles = load_data_file(f)
    break


csv


In [133]:
cols = particles.columns
index = particles[particles[cols[7]]==0].index

In [113]:
len(frames)

4528

In [141]:
# dir(index)

In [140]:
# particles.iloc[0]

In [154]:
%%writefile FuildAINetTest.py

import utils.fluid_loader as fl
BASE_DIR = '/data/datasets/simulation_data'
DATA_DIR = os.path.join(BASE_DIR, 'water')

class TestFluid(object):
    def __init__(self):
        self.data_dir = DATA_DIR
    def test_iter_data(self):
        for item in fl.iterate_data(self.data_dir):
            print(item)
        
if __name__ == '__main__':
    testf1 = TestFluid()
    testf1.test_iter_data()
        

Overwriting FuildAINetTest.py


In [169]:
import utils.fluid_loader as fl
import imp
imp.reload(fl)

<module 'utils.fluid_loader' from '/data/deeplearning/FluidAiNet/utils/fluid_loader.py'>

Fluid
(62032, 3)
2357


In [163]:
fl.iterate_data(DATA_DIR)

<generator object iterate_data at 0x7f6a9a025888>

In [181]:
%%writefile utils/preprocess.py
import os
import multiprocessing
import numpy as np
from nose.tools import assert_equal

from config import cfg

data_dir = 'velodyne'


"""
screen size (-8, -8, -8) (8, 8, 12)
spacing_r 0.1f # diameter 
amoothRadius_h  2*spacing_r # so i decide the voxel_size 4*spacing_r
particle size - radius
x,y,z(21,21,26) (-1, -1, -1) (1, 1, 1.5)
theoreticlly, 
"""

def fluid_process_pointcloud(point_cloud, fluid_identification,cls=cfg.DETECT_OBJ):
    if cls == 'Fluid':
        print('Fluid')
        scene_size = np.array([16, 16, 20], dtype=np.float32)
        voxel_size = np.array([0.4, 0.4, 0.4], dtype=np.float32)
        grid_size = np.array([40, 40, 50], dtype=np.int64)
        lidar_coord = np.array([8, 8, 8], dtype=np.float32)
        max_point_number = 64
        # return
    # FIXME  ccx AWESOME
    print(point_cloud)
    shifted_coord = point_cloud[:, :3] + lidar_coord
    voxel_index = np.floor(
        shifted_coord[:, :] / voxel_size).astype(np.int)  # int lower than num

    print(voxel_index.shape)
    bound_x = np.logical_and(
        voxel_index[:, 2] >= 0, voxel_index[:, 2] < grid_size[2])
    bound_y = np.logical_and(
        voxel_index[:, 1] >= 0, voxel_index[:, 1] < grid_size[1])
    bound_z = np.logical_and(
        voxel_index[:, 0] >= 0, voxel_index[:, 0] < grid_size[0])

    bound_box = np.logical_and(np.logical_and(bound_x, bound_y), bound_z)

    point_cloud = point_cloud[bound_box]
    voxel_index = voxel_index[bound_box]

    # [K, 3] coordinate buffer as described in the paper
    coordinate_buffer = np.unique(voxel_index, axis=0)

    K = len(coordinate_buffer)  # K record the number of voxels
    T = max_point_number
    print(K)
    assert_equal(T, 64)
    # [K, 1] store number of points in each voxel grid
    number_buffer = np.zeros(shape=(K), dtype=np.int64)

    # [K, T, 7] feature buffer as described in the paper
    feature_buffer = np.zeros(shape=(K, T, cfg.VOXEL_POINT_FEATURE), dtype=np.float32) # position, velocity, isFluid, index, relative position

    # build a reverse index for coordinate buffer
    index_buffer = {}
    for i in range(K):
        # mark the voxel,the position tuple of voxel as list(index_buffer)'s key
        index_buffer[tuple(coordinate_buffer[i])] = i

    for voxel, point in zip(voxel_index, point_cloud):
        index = index_buffer[tuple(voxel)]
        number = number_buffer[index]
        if number < T:
            feature_buffer[index, number, :8] = point
            number_buffer[index] += 1

    feature_buffer[:, :, -3:] = feature_buffer[:, :, :3] - \
                                feature_buffer[:, :, :3].sum(axis=1, keepdims=True) / number_buffer.reshape(K, 1, 1)

    voxel_dict = {'feature_buffer': feature_buffer,  # (K, T, 7)
                  'coordinate_buffer': coordinate_buffer,  # (K, 3)
                  'number_buffer': number_buffer}  # (K,)
    return voxel_dict

def process_pointcloud(point_cloud, cls=cfg.DETECT_OBJ):
    # Input:
    #   (N, 4)
    # Output:
    #   voxel_dict
    if cls == 'Car':
        scene_size = np.array([4, 80, 70.4], dtype=np.float32)
        voxel_size = np.array([0.4, 0.2, 0.2], dtype=np.float32)
        grid_size = np.array([10, 400, 352], dtype=np.int64)
        lidar_coord = np.array([0, 40, 3], dtype=np.float32)
        max_point_number = 35
    elif cls == 'Fluid':
        print('Fluid')
        scene_size = np.array([16, 16, 20], dtype=np.float32)
        voxel_size = np.array([0.4, 0.4, 0.4], dtype=np.float32)
        grid_size = np.array([10, 400, 352], dtype=np.int64)
        lidar_coord = np.array([8, 8, 8], dtype=np.float32)
        max_point_number = 64
    else:
        scene_size = np.array([4, 40, 48], dtype=np.float32)
        voxel_size = np.array([0.4, 0.2, 0.2], dtype=np.float32)
        grid_size = np.array([10, 200, 240], dtype=np.int64)
        lidar_coord = np.array([0, 20, 3], dtype=np.float32)# fix coordinate to positive(z,y,x),which is the max num of minus
        max_point_number = 45

        np.random.shuffle(point_cloud)
    # FIXME  ccx AWESOME
    shifted_coord = point_cloud[:, :3] + lidar_coord
    # reverse the point cloud coordinate (X, Y, Z) -> (Z, Y, X)
    voxel_index = np.floor(
        shifted_coord[:, ::-1] / voxel_size).astype(np.int) # int lower than num

    bound_x = np.logical_and(
        voxel_index[:, 2] >= 0, voxel_index[:, 2] < grid_size[2])
    bound_y = np.logical_and(
        voxel_index[:, 1] >= 0, voxel_index[:, 1] < grid_size[1])
    bound_z = np.logical_and(
        voxel_index[:, 0] >= 0, voxel_index[:, 0] < grid_size[0])

    bound_box = np.logical_and(np.logical_and(bound_x, bound_y), bound_z)

    point_cloud = point_cloud[bound_box]
    voxel_index = voxel_index[bound_box]

    # [K, 3] coordinate buffer as described in the paper
    coordinate_buffer = np.unique(voxel_index, axis=0)

    K = len(coordinate_buffer) # K record the number of voxels
    T = max_point_number # TODO ccx set T reasonable?

    # [K, 1] store number of points in each voxel grid
    number_buffer = np.zeros(shape=(K), dtype=np.int64)

    # [K, T, 7] feature buffer as described in the paper
    feature_buffer = np.zeros(shape=(K, T, 7), dtype=np.float32)

    # build a reverse index for coordinate buffer
    index_buffer = {}
    for i in range(K):
        # mark the voxel,the position tuple of voxel as list(index_buffer)'s key
        index_buffer[tuple(coordinate_buffer[i])] = i

    for voxel, point in zip(voxel_index, point_cloud):
        index = index_buffer[tuple(voxel)]
        number = number_buffer[index]
        if number < T:
            feature_buffer[index, number, :4] = point
            number_buffer[index] += 1

    feature_buffer[:, :, -3:] = feature_buffer[:, :, :3] - \
        feature_buffer[:, :, :3].sum(axis=1, keepdims=True)/number_buffer.reshape(K, 1, 1)

    voxel_dict = {'feature_buffer': feature_buffer, # (K, T, 7)
                  'coordinate_buffer': coordinate_buffer, # (K, 3)
                  'number_buffer': number_buffer} # (K,)
    return voxel_dict

if __name__ == "__main__":
#     import fluid_loader

#     BATCH_SIZE = 2
#     TRAIN_FILES = fluid_loader.create_train_files(BATCH_SIZE)
#     print(TRAIN_FILES)
#     pointcloud, _, _ = fluid_loader.concat_data_label_all(TRAIN_FILES, 8, 3)
#     voxel_index = fluid_process_pointcloud(pointcloud[0],1)
    pass

Overwriting utils/preprocess.py


In [182]:
%run -i utils/preprocess.py

In [195]:
BATCH_SIZE = 2
TRAIN_FILES = fl.create_train_files(BATCH_SIZE)
print(TRAIN_FILES)
pointcloud, _, _ = fl.load_data_label(TRAIN_FILES[0])
voxel_index = fluid_process_pointcloud(pointcloud, 1)

['/data/datasets/simulation_data/water/1/particles_665.csv', '/data/datasets/simulation_data/water/1/particles_1504.csv']
csv
Fluid
[[-1.      -1.       1.05335 ..., -8.21249  0.       0.     ]
 [-0.9     -1.       1.05335 ..., -8.21249  0.       0.     ]
 [-0.8     -1.       1.05335 ..., -8.21249  0.       0.     ]
 ..., 
 [ 3.8      4.       3.95    ...,  0.       1.       0.     ]
 [ 3.9      4.       3.95    ...,  0.       1.       0.     ]
 [ 4.       4.       3.95    ...,  0.       1.       0.     ]]
(62032, 3)
2357


AttributeError: 'EasyDict' object has no attribute 'VOXEL_POINT_FEATURE'

In [191]:
import config 

In [193]:
imp.reload(config)

<module 'config' from '/data/deeplearning/FluidAiNet/config.py'>

In [197]:
a = np.array([[1, 2, 3], [4, 5, 6]])

In [198]:
a -[1,1,1]

array([[0, 1, 2],
       [3, 4, 5]])